In [6]:
import pandas as pd
import datetime

In [2]:
userLog = pd.read_csv("./clean_data/userlog_feature.csv")

In [8]:
userLog['datetime'] = pd.to_datetime(userLog['datetime'])
isTrain = userLog['datetime']<datetime.datetime(2018,2,18)
userLog = userLog[isTrain]

In [63]:
userLog1 = userLog.sort_values("datetime")
userLog1.sample()

,userCode,project_id,requestedDevice,userAgent,pageReferrer,year,month,day,hour,datetime,user_clicked_by_hour,user_clicked_projects_by_hour
799106,ed585f1a-51bc-069e-bc07-56560ceb6d84,4816,Desktop,Windows,Google,2018,2,5,9,2018-02-05 09:00:00,3,2


In [17]:
userLog2VecDf = userLog1.groupby("userCode").agg({"project_id":lambda x: list(x)})

In [67]:
userLog2VecDf.sample(10)

,project_id
userCode,
d402e6f2-0cf9-6a3e-c252-e66d9aa01528,"[6391, 7568, 2028]"
273e7f05-8e00-4e6e-836e-ee452946201d,"[6221, 6221]"
306102ba-b2b7-03a4-2e47-4b5f8725ef34,[9084]
120c0645-4914-ee2e-0ce8-2a78dad2bbbc,[6573]
c6e9b2c8-3132-9b35-2651-7283819d26d0,[4716]
85e1b45f-2d6a-4f59-9a20-e9275f939267,"[8611, 8611]"
89b87de8-4d85-7e50-afb9-af8b3cfe5eba,"[8721, 5666]"
a846ea03-f00d-4af7-19b0-d2a1be4a126f,"[8412, 8412, 2219, 2219, 9096, 9096]"
777a5032-89dd-ad17-6b3f-5b7bedc76665,[5945]


In [18]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [42]:
projs = userLog2VecDf['project_id']
users = userLog2VecDf.index

In [43]:
projs2 = [[str(p) for p in proj] for proj in projs if len(proj)>1]

In [48]:
len(projs2)

169721

In [52]:
documents = [TaggedDocument(proj, [user]) for i, (proj,user) in enumerate(zip(projs2,users))]

In [53]:
len(documents)

169721

### Train

In [71]:
model = Doc2Vec(documents, vector_size=300, window=5, min_count=1, workers=4,epochs=20)

### Let's see if it works or not

/home/users/u5722792744_1/miniconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [70]:
most_similar_user = model.docvecs.most_similar('a846ea03-f00d-4af7-19b0-d2a1be4a126f')
#print(most_similar_user)
print(userLog2VecDf.loc[user].project_id)
for user, similarity in most_similar_user:
    print(userLog2VecDf.loc[user].project_id, similarity)

TypeError: '<' not supported between instances of 'str' and 'int'